In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install nltk


In [ ]:
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from collections import defaultdict
import heapq

# Send a GET request to the website
url = 'https://www.moneycontrol.com/news/business/'
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the latest 50 articles
latest_articles = soup.find_all('li', class_='clearfix')[:50]

# Define additional industry types and keywords
keywords = {
    "Automobile": ["car", "automotive", "vehicle", "auto"],
    "Banking": ["bank", "finance", "loan", "financial"],
    "Technology": ["tech", "software", "internet", "innovation"],
    "Healthcare": ["health", "medical", "pharmaceutical", "care"],
    "Energy": ["energy", "oil", "gas", "renewable"],
    "Retail": ["retail", "shopping", "consumer", "store"],
    "Real Estate": ["real estate", "property", "housing", "land"],
    "Food and Beverage": ["food", "beverage", "restaurant", "drink"],
    "Telecommunications": ["telecom", "communication", "network", "wireless"],
    "Entertainment": ["entertainment", "media", "film", "music"],
    "Finance": ["investment", "stock", "trading", "capital"],
    "Insurance": ["insurance", "policy", "coverage", "claim"],
    "Aerospace": ["aerospace", "aviation", "aircraft", "space"],
    "Pharmaceutical": ["pharmaceutical", "drug", "medicine", "clinical"],
    "Manufacturing": ["manufacturing", "factory", "production", "industry"],
    "Transportation": ["transportation", "logistics", "shipment", "cargo"],
    "Hospitality": ["hospitality", "hotel", "travel", "tourism"],
    "Abrasive" : ["abrasive"]
    # Add more industry types and keywords as needed
}

# Create a dictionary to store sentiment polarity for each industry
industry_sentiments = defaultdict(float)

for article in latest_articles:
    # Extract the article URL
    article_url = article.find('a')['href']

    # Extract the headline
    headline = article.find('h2').text.strip()

    try:
        # Send a GET request to the article URL
        article_response = requests.get(article_url)

        # Parse the HTML content of the article page
        article_soup = BeautifulSoup(article_response.content, 'html.parser')

        # Find the relevant element containing the full article content
        article_content = article_soup.find('div', class_='content_wrapper')

        # Verify if article content is found
        if article_content is not None:
            # Extract the full article text
            full_article = article_content.get_text(separator='\n').strip()

            # Perform sentiment analysis on the full article
            article_sentiment = TextBlob(full_article).sentiment
            polarity = article_sentiment.polarity

            # Identify the industry based on the article content using keyword matching
            industry = "Unknown"
            for industry_name, industry_keywords in keywords.items():
                for keyword in industry_keywords:
                    if keyword.lower() in full_article.lower():
                        industry = industry_name
                        break

            # Update the sentiment polarity for the industry
            industry_sentiments[industry] += polarity

            # Highlight sentiment in the full article
            highlighted_article = ''
            for word in full_article.split():
                if TextBlob(word).sentiment.polarity > 0:
                    highlighted_article += '\033[92m' + word + ' '  # Green color for positive words
                elif TextBlob(word).sentiment.polarity < 0:
                    highlighted_article += '\033[91m' + word + ' '  # Red color for negative words
                else:
                    highlighted_article += word + ' '

            # Print the industry, sentiment analysis, headline, link, and the highlighted full article
            print('Industry:', industry)
            print('Sentiment Polarity:', polarity)
            print('Headline:', headline)
            print('Article Link:', article_url)
            print('Full Article:', highlighted_article)
            print('--------------------------------------')

    except Exception as e:
        print('Error occurred while analyzing the article:', headline)
        print('Error:', str(e))
        print('--------------------------------------')

# Get the top 5 industries with the highest sentiment polarity
top_industries = heapq.nlargest(5, industry_sentiments, key=industry_sentiments.get)

print('\nTop 5 Bullish Industries:')
for industry in top_industries:
    sentiment_score = industry_sentiments[industry]
    print('Industry:', industry)
    print('Sentiment Score:', sentiment_score)
    print('--------------------------------------')


Industry: Insurance
Sentiment Polarity: 0.08538961038961039
Headline: Hold Dalmia Bharat; target of Rs 2230: Axis Securities
Article Link: https://www.moneycontrol.com/news/business/stocks/hold-dalmia-bharat-target-of-rs-2230-axis-securities-10953781.html
Full Article: hold Trade Watchlist Portfolio Message Set Alert live bse live nse live Volume Todays L/H More × Axis Securities report on Dalmia Bharat Dalmia Bharat Limited (DBL) –is one of the established cement player sin India. The company expanded its cement capacity from 35.9 MTPA to 38.6 MTPA. In Apr’23, the company also started operations ata second cement line in Bokaro, Jharkhand, with a capacity of 2.5 MTPA. It also signed the definitive agreement for the acquisition of the strategically located cement assets of Jai prakash Associates Limited, (9.4 mtpa Grinding and 6.7 mtpa Clinker capacity at an enterprise value of Rs5,670 Cr). This will enable the company to enter the lucrative growth market of central India. DBL aims to 